# Build an AI agent from scratch using Llama3:70b (served via Ollama)

Based off on the paper: https://arxiv.org/pdf/2210.03629 

### Importing Necessary Libraries

In [1]:
import openai
import re
import httpx
import os
from dotenv import load_dotenv

_ = load_dotenv()
from openai import OpenAI

### Instantiating the OpenAI Client

Ollama provides experimental compatibility with parts of the OpenAI API to help connect existing applications to Ollama.

In [2]:
from openai import OpenAI

client = OpenAI(
    base_url='http://localhost:11434/v1/',

    # required but ignored
    api_key='ollama',
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            'role': 'user',
            'content': 'Say this is a test',
        }
    ],
    model='llama3:70b',
)

In [3]:
chat_completion.choices[0].message.content

"It looks like you're testing to see if I'm working properly! That's perfectly fine. I'm happy to report that I'm functioning as intended and ready to assist with any questions or topics you'd like to discuss. How can I help you today?"

### Creating an Agent using class

In [4]:
class Agent:
    def __init__(self, system=""):
        self.system = system
        self.messages = []
        if self.system:
            self.messages.append({"role": "system", "content": system})

    def __call__(self, message):
        self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result

    def execute(self):
        completion = client.chat.completions.create(
                        model="llama3:70b", 
                        temperature=0,
                        messages=self.messages)
        return completion.choices[0].message.content
    

### `ReAct` Prompt

In [5]:
prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

average_dog_weight:
e.g. average_dog_weight: Collie
returns average weight of a dog when given the breed

Example session:

Question: How much does a Bulldog weigh?
Thought: I should look the dogs weight using average_dog_weight
Action: average_dog_weight: Bulldog
PAUSE

You will be called again with this:

Observation: A Bulldog weights 51 lbs

You then output:

Answer: A bulldog weights 51 lbs
""".strip()

Note that these actions can be changed as per our usecases. Later, we will see dynamic and flexible `reAct` prompt for it that won't require us to change the actions always.

In [6]:
def calculate(what):
    return eval(what)

# To evaluate a string-based expression, Python’s eval() runs the following steps:

# Parse expression
# Compile it to bytecode
# Evaluate it as a Python expression
# Return the result of the evaluation

# Source: https://realpython.com/python-eval-function/

def average_dog_weight(name):
    if name in "Scottish Terrier": 
        return("Scottish Terriers average 20 lbs")
    elif name in "Border Collie":
        return("a Border Collies average weight is 37 lbs")
    elif name in "Toy Poodle":
        return("a toy poodles average weight is 7 lbs")
    else:
        return("An average dog weights 50 lbs")

known_actions = {
    "calculate": calculate,
    "average_dog_weight": average_dog_weight
}

#### Doing it manually without loop

In [7]:
abot = Agent(prompt)

In [8]:
result = abot("How much does a toy poodle weigh?")
print(result)

Thought: I should look up the dog's weight using average_dog_weight.

Action: average_dog_weight: Toy Poodle

PAUSE


In [9]:
result = average_dog_weight("Toy Poodle")

In [10]:
result

'a toy poodles average weight is 7 lbs'

In [11]:
next_prompt = "Observation: {}".format(result)

In [12]:
abot(next_prompt)

'Answer: A toy poodle weighs 7 lbs.'

In [13]:
abot.messages

[{'role': 'system',
  'content': 'You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\naverage_dog_weight:\ne.g. average_dog_weight: Collie\nreturns average weight of a dog when given the breed\n\nExample session:\n\nQuestion: How much does a Bulldog weigh?\nThought: I should look the dogs weight using average_dog_weight\nAction: average_dog_weight: Bulldog\nPAUSE\n\nYou will be called again with this:\n\nObservation: A Bulldog weights 51 lbs\n\nYou then output:\n\nAnswer: A bulldog weights 51 lbs'},
 {'role': 'user', 'content': 'How much does a 

In [14]:
abot = Agent(prompt)

In [15]:
question = """I have 2 dogs, a border collie and a scottish terrier. \
What is their combined weight"""
abot(question)

'Thought: I should look up the average weights of both breeds using average_dog_weight.\n\nAction: average_dog_weight: Border Collie\nPAUSE'

In [16]:
next_prompt = "Observation: {}".format(average_dog_weight("Border Collie"))
print(next_prompt)

Observation: a Border Collies average weight is 37 lbs


In [17]:
abot(next_prompt)

'Thought: Now that I have the weight of the Border Collie, I should look up the average weight of the Scottish Terrier.\nAction: average_dog_weight: Scottish Terrier\nPAUSE'

In [18]:
next_prompt = "Observation: {}".format(average_dog_weight("Scottish Terrier"))
print(next_prompt)

Observation: Scottish Terriers average 20 lbs


In [19]:
next_prompt = "Observation: {}".format(average_dog_weight("Scottish Terrier"))
print(next_prompt)

Observation: Scottish Terriers average 20 lbs


In [20]:
next_prompt = "Observation: {}".format(eval("37 + 20"))
print(next_prompt)

Observation: 57


#### Adding loop

In [21]:
action_re = re.compile('^Action: (\w+): (.*)$')   # python regular expression to selection action

In [22]:
def query(question, max_turns=5):
    i = 0
    bot = Agent(prompt)
    next_prompt = question
    while i < max_turns:
        i += 1
        result = bot(next_prompt)
        print(result)
        actions = [
            action_re.match(a) 
            for a in result.split('\n') 
            if action_re.match(a)
        ]
        if actions:
            # There is an action to run
            action, action_input = actions[0].groups()
            if action not in known_actions:
                raise Exception("Unknown action: {}: {}".format(action, action_input))
            print(" -- running {} {}".format(action, action_input))
            observation = known_actions[action](action_input)
            print("Observation:", observation)
            next_prompt = "Observation: {}".format(observation)
        else:
            return

In [23]:
question = """I have 2 dogs, a border collie and a scottish terrier. \
What is their combined weight"""
query(question)

Thought: I should look up the average weights of both breeds using average_dog_weight.

Action: average_dog_weight: Border Collie
PAUSE
 -- running average_dog_weight Border Collie
Observation: a Border Collies average weight is 37 lbs
Thought: Now that I have the weight of the Border Collie, I should look up the average weight of the Scottish Terrier.
Action: average_dog_weight: Scottish Terrier
PAUSE
 -- running average_dog_weight Scottish Terrier
Observation: Scottish Terriers average 20 lbs
Thought: Now that I have the weights of both breeds, I can calculate their combined weight.
Action: calculate: 37 + 20
PAUSE
 -- running calculate 37 + 20
Observation: 57
Answer: My two dogs, a Border Collie and a Scottish Terrier, weigh a combined 57 lbs.
